In [42]:
%matplotlib inline

import glob
import os

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

In [73]:
TEST_CASES_PATH = "results_remote_lb"

TEST_CASES_DICT = {
    'Baseline': '2019_04_04_161815',
    'Default Scaling': '2019_04_04_203319',
    'Low Threshold Scaling': '2019_04_04_233925',
    'Aggressive Scaling': '2019_04_05_145551'
}

FILE_TYPE_AWS_METRICS = 'aws_metrics_{0}_*.csv'
FILE_TYPE_USER_RESPONSE = 'users{1}_response_{0}.csv'

SAMPLE_PERIOD_SECONDS = 30

def read_file(test_case_name, file_type, *args):
    file_name = file_type.format(*args)
    file_search_path = os.path.join(TEST_CASES_PATH, TEST_CASES_DICT[test_case_name], file_name)
    file_paths = glob.glob(file_search_path)
    
    # Get test file with test params if not searching aws metrics
    if file_type != FILE_TYPE_AWS_METRICS:
        s_file_name = FILE_TYPE_AWS_METRICS.format(args[0])
        s_file_search_path = os.path.join(TEST_CASES_PATH, TEST_CASES_DICT[test_case_name], s_file_name)
        s_file_paths = glob.glob(s_file_search_path)
        info_file = s_file_paths[0]
    else:
        info_file = file_paths[0]
        
    params_list = os.path.basename(info_file)[12:].split('_')
    
    params_dict = {
        'start_time_seconds': int(params_list[1]),
        'end_time_seconds': int(params_list[2])
    }

    return pd.read_csv(file_paths[0]), params_dict

def get_user_responses_for_test_case(test_case_name, user='A'):
    user_responses_list = []
    for i in range(3):
        df, params_dict = read_file(test_case_name, FILE_TYPE_USER_RESPONSE, i, user)
        df.loc[:,'timepoint'] = (df.loc[:,'timeStamp'] / 1000) - params_dict['start_time_seconds']
        user_responses_list.append(df)
    return user_responses_list

# using this method instead of np.std because np.std divides sum of squares by n (biased sample variance), not n-1
def get_standard_deviation(mean, values):
    n = len(values)
    s = 0
    for i in range(n):
        s += (values[i] - mean) ** 2
    
    return np.sqrt(s / (n-1))

def get_CI_for_response_time_difference_for_user(system_1, system_2, user):
    n = 3
    user_responses_list_system_1 = get_user_responses_for_test_case(system_1, user)
    user_responses_list_system_2 = get_user_responses_for_test_case(system_2, user)
    differences = []
    
    for i in range(n):
        mean_system_1 = np.mean(user_responses_list_system_1[i]['elapsed']) / 1000
        mean_system_2 = np.mean(user_responses_list_system_2[i]['elapsed']) / 1000
        differences.append(mean_system_1 - mean_system_2)
        
    sample_mean = np.mean(differences)
    sample_std = get_standard_deviation(sample_mean, differences)
    # alpha = 0.05, we want t statistic at .975, 2
    t = stats.t.ppf(.975, n-1)
    interval_half_size = t * sample_std / np.sqrt(n)
    confidence_interval = [sample_mean - interval_half_size, sample_mean + interval_half_size]
    
    print('''The 95% confidence interval for the difference in mean response times between {0} and {1} for user class {2} is ({3}, {4})'''.format(system_1, system_2, user, confidence_interval[0], confidence_interval[1]))
    print()
    
def get_CI_for_response_time_difference(system_1, system_2):
    get_CI_for_response_time_difference_for_user(system_1, system_2, 'A')
    get_CI_for_response_time_difference_for_user(system_1, system_2, 'B')
    get_CI_for_response_time_difference_for_user(system_1, system_2, 'C')

In [74]:
get_CI_for_response_time_difference('Baseline', 'Default Scaling')

The 95% confidence interval for the difference in mean response times between Baseline and Default Scaling for user class A is (6.962075604101352, 12.40579745021525)

The 95% confidence interval for the difference in mean response times between Baseline and Default Scaling for user class B is (12.152109440173314, 13.340549956532316)

The 95% confidence interval for the difference in mean response times between Baseline and Default Scaling for user class C is (11.531901981968616, 13.13597272935641)



In [75]:
get_CI_for_response_time_difference('Baseline', 'Low Threshold Scaling')

The 95% confidence interval for the difference in mean response times between Baseline and Low Threshold Scaling for user class A is (4.552905998665879, 13.609140924301798)

The 95% confidence interval for the difference in mean response times between Baseline and Low Threshold Scaling for user class B is (10.513346787642316, 14.44576605556745)

The 95% confidence interval for the difference in mean response times between Baseline and Low Threshold Scaling for user class C is (9.934600389210939, 13.793491837768038)



In [76]:
get_CI_for_response_time_difference('Baseline', 'Aggressive Scaling')

The 95% confidence interval for the difference in mean response times between Baseline and Aggressive Scaling for user class A is (9.582963967524963, 10.791582616676914)

The 95% confidence interval for the difference in mean response times between Baseline and Aggressive Scaling for user class B is (11.911802571596562, 14.834477939796377)

The 95% confidence interval for the difference in mean response times between Baseline and Aggressive Scaling for user class C is (10.391347488125763, 15.465122091318602)



In [77]:
get_CI_for_response_time_difference('Default Scaling', 'Low Threshold Scaling')

The 95% confidence interval for the difference in mean response times between Default Scaling and Low Threshold Scaling for user class A is (-2.7947106703988864, 1.588884539049959)

The 95% confidence interval for the difference in mean response times between Default Scaling and Low Threshold Scaling for user class B is (-1.7462102516084725, 1.2126636981126089)

The 95% confidence interval for the difference in mean response times between Default Scaling and Low Threshold Scaling for user class C is (-2.1702496780095224, 1.2304671936634757)



In [78]:
get_CI_for_response_time_difference('Default Scaling', 'Aggressive Scaling')

The 95% confidence interval for the difference in mean response times between Default Scaling and Aggressive Scaling for user class A is (-1.7407456201039047, 2.74741914998918)

The 95% confidence interval for the difference in mean response times between Default Scaling and Aggressive Scaling for user class B is (-1.2958232934021408, 2.549444408089449)

The 95% confidence interval for the difference in mean response times between Default Scaling and Aggressive Scaling for user class C is (-1.3397344422788957, 2.5283293103982354)



In [79]:
get_CI_for_response_time_difference('Low Threshold Scaling', 'Aggressive Scaling')

The 95% confidence interval for the difference in mean response times between Low Threshold Scaling and Aggressive Scaling for user class A is (-2.836221385457816, 5.048721046692018)

The 95% confidence interval for the difference in mean response times between Low Threshold Scaling and Aggressive Scaling for user class B is (-1.8322924110641097, 3.6194600792472817)

The 95% confidence interval for the difference in mean response times between Low Threshold Scaling and Aggressive Scaling for user class C is (-2.290258882607489, 4.418636235072875)

